# Playing with Microsoft Graph Security (via REST API and APP)
(https://docs.microsoft.com/en-us/graph/api/resources/security-api-overview?view=graph-rest-1.0)

- Requirement: the APP should have the following 'API permissions': 
 - Microsoft Graph:
   - SecurityEvents.Read.All (for reading alarms, secure scores, secureScoreControlProfiles)
   - ThreatAssessment.Read.All (for reading threat assessments)

<h1 align='center'>=======================================================<br>=======================================================</h1>

# Required information

In [ ]:
appId = '' 
tenantId = ''
appSecret = '' 
subscription_id = '' #zuidgolf

# Required Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import urllib.parse
pd.set_option('display.max_columns', None)

<h1 align='center'>=======================================================<br>=======================================================</h1>

# Function to get the token to access via the app

In [ ]:
def getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://graph.microsoft.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

# Instantiating previous function

In [ ]:
access_token = getaadtoken(tenantId, appId, appSecret)

<h1 align='center'>=======================================================<br>=======================================================</h1>

# Function to get the list of alerts
https://docs.microsoft.com/en-us/rest/api/azureresourcegraph/resourcegraph(2019-04-01)/resources

In [ ]:
def securitygraph_listalerts(access_token):
    url = "https://graph.microsoft.com/v1.0/security/alerts"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating previous function

In [ ]:
df_alerts = securitygraph_listalerts(access_token)

# Extending the provider name

In [ ]:
df_alerts['provider'] = df_alerts['vendorInformation'].apply(lambda x: x['provider'])

# How many alerts are related to what security provider?

In [ ]:
df_alerts['provider'].value_counts()

<h1 align='center'>========================================================<br>=======================================================<br>INFORMATION PROTECTION: THREAT ASSESSMENT API</h1>

https://docs.microsoft.com/en-us/graph/api/resources/threatassessment-api-overview?view=graph-rest-1.0

|Method|	Return Type|	Description|
|:-|:-|:-|
|List threatAssessmentRequest|	threatAssessmentRequest collection|	List all threat assessment requests under tenant.|
|Create threatAssessmentRequest|	threatAssessmentRequest	|Create a new threat assessment request by posting a derived resource type: mailAssessmentRequest, emailFileAssessmentRequest, fileAssessmentRequest, urlAssessmentRequest.|
|Get threatAssessmentRequest|	threatAssessmentRequest|	Retrieve the properties and relationships of a specified threatAssessmentRequest resource.|

# Function to get the list o threat assessment requested

In [ ]:
def securitygraph_listthreatassessmentrequest(access_token):
    url = "https://graph.microsoft.com/v1.0/informationProtection/threatAssessmentRequests"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
df_threatassessmentrequests = securitygraph_listthreatassessmentrequest(access_token)

# What is the list of requested threat assessments?

In [ ]:
df_threatassessmentrequests

<h1 align='center'>========================================================<br>=======================================================<br>SECURE SCORES API</h1>

https://docs.microsoft.com/en-us/graph/api/resources/securescore?view=graph-rest-1.0

|Method|	Return Type|	Description|
|:-|:-|:-|
|List secureScores|	secureScores collection|	Get secureScore object collection.|
|Get secureScore|	secureScore|	Read properties and metadata of a secureScore object.

# Function list secure scores

In [ ]:
def securitygraph_listsecurescores(access_token):
    url = "https://graph.microsoft.com/v1.0/security/secureScores"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
df_securescores = securitygraph_listsecurescores(access_token)

# How the secure score looks like?

In [ ]:
df_securescores.head(2)

In [ ]:
pd.set_option('display.max_colwidth', 30)

# Detailing 1 day of securite scores

In [ ]:
securescore_lastday = pd.DataFrame(df_securescores['controlScores'][0])

In [ ]:
securescore_lastday

**NOTE:** Based on the previous table, the secure score is based on 32 items

In [ ]:
securescore_lastday['controlCategory'].value_counts()

<h1 align='center'>========================================================<br>=======================================================<br>SECURE SCORE CONTROL PROFILES API</h1>

https://docs.microsoft.com/en-us/graph/api/resources/securescorecontrolprofile?view=graph-rest-1.0

|Method|	Return Type|	Description|
|:-|:-|:-|
|List secureScoreControlProfiles|	secureScoreControlProfile|	Read properties and metadata of a secureScoreControlProfiles object.|
|Get secureScoreControlProfile|	securescorecontrolprofile|	Read properties and metadata of a secureScoreControlProfiles object.|
|Update securescorecontrolprofile|	securescorecontrolprofile|	Update an securescorecontrolprofile object.|

    

# Function to get a list of secure score control profiles

In [ ]:
def securitygraph_listsecurescorecontrolprofiles(access_token):
    url = "https://graph.microsoft.com/v1.0/security/secureScoreControlProfiles"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

In [ ]:
df_securescorecontrolprofiles = securitygraph_listsecurescorecontrolprofiles(access_token)

In [ ]:
df_securescorecontrolprofiles